In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
model_name = "loss_proxy"
agents_to_sample = [1, 10, 20, 30, 40, 100, 150, 200]
full_name = "5x4silverman"
dims = (5,4)
np.seterr(over="ignore", invalid="raise")


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
import tensorflow as tf
from minichess.agents.lite_model import LiteModel
from minichess.agents.predictor_convnet import PredictorConvNet

def load_model(full_name, model_name, epoch):
    keras_model = tf.keras.models.load_model("minichess/agents/checkpoints/{}/{}/{}".format(full_name, model_name, epoch))
    simple_model = PredictorConvNet(LiteModel.from_keras_model(keras_model))
    del keras_model
    return simple_model

agents = [load_model(full_name, model_name, epoch) for epoch in agents_to_sample]

INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmphcmar309\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmphcmar309\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmp19a1f3k6\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmp19a1f3k6\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmplrwcbfdh\assets


...

INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmpe1bikvj2\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmpmzd14okc\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmpmzd14okc\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmpsaptt00o\assets


INFO:tensorflow:Assets written to: C:\Users\Patrik\AppData\Local\Temp\tmpsaptt00o\assets


In [4]:
from minichess.agents.lite_model import LiteModel
from minichess.agents.predictor_convnet import PredictorConvNet
from minichess.chess.fastchess import Chess
from minichess.chess.fastchess_utils import piece_matrix_to_legal_moves, visualize_board
from minichess.chess.move_utils import calculate_all_moves, index_to_move, move_to_index
from minichess.rl.chess_helpers import get_initial_chess_object
import numpy as np

import tensorflow as tf
def play_match(agents, full_name, dims, move_cap, all_moves, all_moves_inv, concept_function):
    chess = get_initial_chess_object(full_name)
    to_start = 1 if np.random.random() > 0.5 else 0
    current = to_start
    positive_cases = []
    negative_cases = []
    SAMPLING_RATIO = 0.2

    while chess.game_result() is None:
        if np.random.random() < SAMPLING_RATIO:
            if concept_function(chess):
                positive_cases.append(chess.agent_board_state())
            else:
                negative_cases.append(chess.agent_board_state())


        agent_to_play = agents[current]
        dist, value = agent_to_play.predict(chess.agent_board_state())

        moves, proms = chess.legal_moves()
        legal_moves = piece_matrix_to_legal_moves(moves, proms)
        legal_moves_mask = np.zeros((dims[0], dims[1], all_moves_inv.shape[0]))
        for move in legal_moves:
            (i, j), (dx, dy), promotion = move
            ind = move_to_index(all_moves, dx, dy, promotion, chess.turn)
            legal_moves_mask[i, j, ind] = 1

        move_dims = dist.shape

        dist = (dist + 0.5 * np.random.uniform(size=dist.shape)) * legal_moves_mask.flatten()

        dist /= dist.sum()
        move_to_play = np.argmax(dist)

        # move_to_play = np.random.choice(np.arange(dist.shape[0]), p=dist)
        i, j, ind = np.unravel_index(move_to_play, (dims[0], dims[1], move_cap))
        dx, dy, promotion = index_to_move(all_moves_inv, ind, chess.turn)
        chess.make_move(i, j, dx, dy, promotion)
        current = (current + 1) % 2
    return positive_cases, negative_cases

In [5]:
all_moves, all_moves_inv = calculate_all_moves(dims)
move_cap = all_moves_inv.shape[0]

In [6]:
from minichess.concepts.concepts import in_check, random, threat_opp_queen, material_advantage, has_mate_threat, opponent_has_mate_threat, has_contested_open_file
CONCEPT_FUNC = has_contested_open_file
concept_name = "has_contested_open_file"


In [7]:
from tqdm import tqdm
positive_cases = []
negative_cases = []

CASES_TO_COLLECT = 25000
pbar = tqdm(total=CASES_TO_COLLECT)
while len(positive_cases) < CASES_TO_COLLECT:
    pos, neg = play_match([agents[0], agents[2]], full_name, dims, move_cap, all_moves, all_moves_inv, CONCEPT_FUNC)
    positive_cases.extend(pos)
    negative_cases.extend(neg)
    pbar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_COLLECT]
negative_cases = negative_cases[:CASES_TO_COLLECT]

100%|█████████▉| 24997/25000 [13:04<00:00, 37.71it/s]

In [8]:
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [9]:
positive_cases.shape[0]

25000

In [10]:
negative_cases.shape[0]

25000

In [11]:
epochs_to_look_at = [1, 10, 20, 30, 40, 100, 150, 200]

In [12]:
from minichess.agents.convnet import ConvNet

for epoch_to_look_at in epochs_to_look_at:
    import tensorflow.keras as keras
    predictor_model = ConvNet(None, None, init=False)
    predictor_model.model = keras.models.load_model("minichess/agents/checkpoints/{}/{}/{}".format(full_name, model_name, epoch_to_look_at))
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])
    np.random.shuffle(shuffled_indices)
    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]
    POSITIONS_TO_CONSIDER = 40000
    VALIDATION_POSITIONS = 10000
    from minichess.concepts.linear_regression import perform_linear_regression, perform_logistic_regression, perform_regression

    concept_presences = {}


    print("Getting outputs...")
    outputs = predictor_model.get_all_resblock_outputs(all_cases)
    # actual_outputs = predictor_model.predict(boards, id_vector_to_use)
    # Outputs blir returnert i batcher, må flette det sammen
    print("Merging outputs...")
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    outputs = merged_outputs
    print("Outputs merged.")
    # Aktiveringer fra res-block i
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        print(points.shape)
        # Så man har (n, k) sampler der n er antallet posisjoner, og k er det totale antallet aktiveringsverdier i lag i.
        print("Performing regression for layer {}".format(i))
        # points = np.concatenate([points, actual_outputs], axis=1)
        score = perform_regression(
            points[:POSITIONS_TO_CONSIDER], 
            all_labels[:POSITIONS_TO_CONSIDER], 
            points[POSITIONS_TO_CONSIDER:], 
            all_labels[POSITIONS_TO_CONSIDER:], 
            True
        )
        concept_presence_per_layer.append(score)

        print("The presence of {} in resblock {} is {}".format(concept_name, i, score))
    concept_presences[concept_name] = concept_presence_per_layer
    import os
    import string
    from random import choices
    import json



    os.makedirs("concept_presences", exist_ok=True)
    os.makedirs("concept_presences/{}".format(full_name), exist_ok=True)
    os.makedirs("concept_presences/{}/{}".format(full_name, model_name), exist_ok=True)
    os.makedirs("concept_presences/{}/{}/{}".format(full_name, model_name, concept_name), exist_ok=True)
    os.makedirs("concept_presences/{}/{}/{}/{}".format(full_name, model_name, concept_name, epoch_to_look_at), exist_ok=True)

    random_suffix = ''.join(choices(string.ascii_uppercase + string.digits, k=10))

    with open("concept_presences/{}/{}/{}/{}/{}.json".format(full_name, model_name, concept_name, epoch_to_look_at, random_suffix), "w") as f:
        json.dump(concept_presences[concept_name], f)


Getting outputs...


100%|██████████| 1563/1563 [00:07<00:00, 216.17it/s]


Merging outputs...
Outputs merged.
(50000, 640)
Performing regression for layer 0
Epoch 1/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.8071 - val_loss: 0.6997
Epoch 2/30
 996/1250 [======================>.......] - ETA: 0s - loss: 0.6951

100%|██████████| 25000/25000 [13:20<00:00, 37.71it/s]

1250/1250 [==============================] - 4s 3ms/step - loss: 0.6948 - val_loss: 0.6934
Epoch 3/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6935 - val_loss: 0.6934
Epoch 4/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 5/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 6/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 7/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 8/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 9/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 10/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 11/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6934 - val_lo

...

Merging outputs...
Outputs merged.
(50000, 640)
Performing regression for layer 0
Epoch 1/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.9310 - val_loss: 0.8264
Epoch 2/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.7615 - val_loss: 0.7202
Epoch 3/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6868 - val_loss: 0.6665
Epoch 4/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6485 - val_loss: 0.6386
Epoch 5/30
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6289 - val_loss: 0.6246
Epoch 6/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6177 - val_loss: 0.6154
Epoch 7/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6110 - val_loss: 0.6105
Epoch 8/30
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6068 - val_loss: 0.6068
Epoch 9/30
1250/1250 [==============================] - 3s 3ms/step - loss: 0.6041 - val_loss: 0.6041


100%|██████████| 1563/1563 [00:07<00:00, 213.53it/s]


Merging outputs...
Outputs merged.
(50000, 640)
Performing regression for layer 0
Epoch 1/30
1250/1250 [==============================] - 7s 5ms/step - loss: 0.9408 - val_loss: 0.8299
Epoch 2/30
1250/1250 [==============================] - 6s 5ms/step - loss: 0.7676 - val_loss: 0.7212
Epoch 3/30
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6874 - val_loss: 0.6617
Epoch 4/30
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6436 - val_loss: 0.6287
Epoch 5/30
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6217 - val_loss: 0.6139
Epoch 6/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6100 - val_loss: 0.6063
Epoch 7/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6037 - val_loss: 0.5995
Epoch 8/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5999 - val_loss: 0.5956
Epoch 9/30
1250/1250 [==============================] - 5s 4ms/step - loss: 0.5971 - val_loss: 0.5938


100%|██████████| 1563/1563 [00:05<00:00, 272.19it/s]


Merging outputs...
Outputs merged.
(50000, 640)
Performing regression for layer 0
Epoch 1/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.9338 - val_loss: 0.8065
Epoch 2/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.7502 - val_loss: 0.7026
Epoch 3/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6722 - val_loss: 0.6461
Epoch 4/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6313 - val_loss: 0.6182
Epoch 5/30
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6101 - val_loss: 0.6047
Epoch 6/30
1250/1250 [==============================] - 5s 4ms/step - loss: 0.5982 - val_loss: 0.5945
Epoch 7/30
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5915 - val_loss: 0.5899
Epoch 8/30
1250/1250 [==============================] - 5s 4ms/step - loss: 0.5873 - val_loss: 0.5879
Epoch 9/30
1250/1250 [==============================] - 5s 4ms/step - loss: 0.5851 - val_loss: 0.5856
